In [14]:
!pip install PyPDF2



In [15]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0
#install required packages
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio pypdf
!pip install gradio

In [16]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')
import gradio as gr
from llama_index.core import SimpleDirectoryReader

In [4]:
from google.colab import userdata
GROQ_API_KEY ="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"

In [9]:
def embedding(documents):
    # documents = SimpleDirectoryReader(
    #     input_files=[file_name]
    # ).load_data()
    #documents = file_name
    text_splitter = SentenceSplitter(chunk_size=4000, chunk_overlap=200)
    nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)
    embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
    llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)
    #print("Document ID:", documents[0].doc_id)
    service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
    vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)
    vector_index.storage_context.persist(persist_dir="./storage_mini1")
    storage_context = StorageContext.from_defaults(persist_dir="./storage_mini1")
    index = load_index_from_storage(storage_context, service_context=service_context)
    query_engine = index.as_query_engine(service_context=service_context)
    return query_engine

In [17]:
documents = SimpleDirectoryReader(
    input_files=["/content/Credit_Card_Fraud_Detection_Model_Based.pdf"]
).load_data()

[Document(id_='81bf405e-44e4-4c9c-a059-38fefafbfdbb', embedding=None, metadata={'page_label': '1', 'file_name': 'Credit_Card_Fraud_Detection_Model_Based.pdf', 'file_path': '/content/Credit_Card_Fraud_Detection_Model_Based.pdf', 'file_type': 'application/pdf', 'file_size': 430667, 'creation_date': '2024-08-13', 'last_modified_date': '2024-08-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \nCredit Card Fraud Detection Model Based on \nLSTM Recurrent Neural Networks \n \nIbtissam Benchaji, Samira Douzi, and Bouabid El Ouahidi \nFaculty of Sciences IPSS, University Mohammed V, Rabat, Morocco \nEmail: ibtissam_benchaji@um5.ac.ma, {samiradouzi8 , Bouabid.ouahidi}@gmail.com \n \n \n \nAbstract—With the increasing use of credit cards in \nelectronic paym

In [22]:
# query_engine=embedding(extracted_text)
# query_s = "based on the provided content you should provide the answer"
# query="provide the abstract of this patent"
# query = query + query_s
# # Assuming query_engine.query(query) is defined elsewhere
# resp = query_engine.query(query)

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/12 [00:00<?, ?it/s]

In [23]:
print(resp)

Based on the provided content, the authors have contributed to the field of credit card fraud detection by proposing various approaches and techniques. 

A. C. Bahnsen has contributed to feature engineering strategies for credit card fraud detection, cost-sensitive credit card fraud detection using Bayes minimum risk, and has provided a practitioner's perspective on expert systems with applications.

V. V. Vlasselaer has contributed to a novel approach for automated credit card transaction fraud detection using network-based extensions.

V. R. Ganji has contributed to credit card fraud detection using an anti-k nearest neighbor algorithm.

J. Pun has contributed to improving credit card fraud detection using a meta-classification strategy.

F. T. Liu has contributed to the concept of isolation forest.

X. Zhao has contributed to a local outlier mining algorithm based on attribute relevance analysis.

C. S. Hemalatha has contributed to a minimal infrequent pattern-based approach for min

In [25]:
import gradio as gr
def answer_query(pdf_path, query):
    # Use SimpleDirectoryReader to load the PDF document
    documents = SimpleDirectoryReader(
        input_files=[pdf_path]
    ).load_data()
    query_engine=embedding(documents)
    query_s = "based on the provided content you should provide the answer"
    #query="provide the contribution of Author"
    query = query + query_s
    # Assuming query_engine.query(query) is defined elsewhere
    resp = query_engine.query(query)


    return resp

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# PDF Query Answering")

    with gr.Row():
        with gr.Column():
            pdf_file = gr.File(label="Upload PDF")
        with gr.Column():
            query = gr.Textbox(label="Enter your query")

    submit_btn = gr.Button("Submit")
    output = gr.Textbox(label="Response")

    submit_btn.click(
        fn=answer_query,
        inputs=[pdf_file, query],
        outputs=output
    )

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://07a552d26977a8b925.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
